# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

<div class="alert alert-block alert-warning">
<b>Note:</b>
As of v0.5, most outputs, including availability, return values as a pandas DataFrame, regardless of the aggregation level.
</div>


In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [5]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
print(f"Project total: {total * 100:.1f}%")

Project total: 96.1%


In [6]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.87,0.94,0.98,0.96,0.98,...,0.93,0.98,0.98,0.98,0.86


In [7]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.96
2004,0.95
2005,0.96
2006,0.96
2007,0.97
2008,0.96
2009,0.97
2010,0.96
2011,0.96


In [8]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.96
2,0.96
3,0.96
4,0.95
5,0.96
6,0.97
7,0.97
8,0.96
9,0.96


In [9]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.99
     2          0.97
     3          0.97
     4          0.97
     5          0.95
...              ...
2012 8          0.97
     9          0.96
     10         0.95
     11         0.95
     12         0.95

[120 rows x 1 columns]

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [15]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.46%
Gross Capacity Factor: 0.48%


In [16]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.42,0.45,0.47,0.46,0.47,...,0.44,0.47,0.46,0.47,0.41


In [19]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.40
2004,0.44
2005,0.47
2006,0.45
2007,0.49
2008,0.50
2009,0.46
2010,0.41
2011,0.49


In [20]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.60
2,0.51
3,0.50
4,0.41
5,0.39
6,0.32
7,0.33
8,0.36
9,0.46


In [22]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.63
     2          0.36
     3          0.37
     4          0.52
     5          0.34
...              ...
2012 8          0.31
     9          0.54
     10         0.51
     11         0.52
     12         0.60

[120 rows x 1 columns]

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [26]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 100%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [34]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,0.99
2004,0.98
2005,1.02
2006,1.00
2007,1.01
2008,0.99
2009,1.00
2010,0.99
2011,1.00


In [35]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.94
2,1.03
3,1.00
4,1.01
5,1.04
6,1.01
7,0.98
8,0.98
9,0.99


In [37]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year")

Completion Rate
year month                 
2003 1                 0.89
     2                 1.03
     3                 0.92
     4                 1.09
     5                 1.04
...                     ...
2012 8                 0.95
     9                 1.01
     10                0.97
     11                1.00
     12                1.04

[120 rows x 1 columns]

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [38]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $461,396.45/MW


In [39]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,393,260.42","6,393,260.42","6,393,260.42","1,303,653.99","90,251,713.54"


In [40]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"9,918,178.67"
2004,"16,347,568.97"
2005,"7,012,759.95"
2006,"15,626,456.25"
2007,"10,862,081.96"
2008,"7,034,253.47"
2009,"12,146,461.83"
2010,"7,498,282.99"
2011,"12,734,616.52"


In [41]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"543,010.42","543,010.42","543,010.42","228,000.00","16,289,208.33"
2,"496,125.00","495,250.00","495,250.00","76,633.88","5,420,312.50"
3,"541,989.58","542,500.00","542,864.58","14,250.00","1,183,652.78"
4,"524,635.42","525,000.00","524,635.42","31,206.62","13,293,052.08"
5,"542,500.00","542,500.00","542,500.00","209,810.14","13,977,862.85"
6,"525,000.00","525,000.00","525,000.00","323,998.60","8,411,722.22"
7,"542,500.00","542,500.00","542,500.00","19,000.00","4,800,000.00"
8,"542,500.00","542,500.00","542,500.00",0.00,"4,950,180.56"
9,"525,000.00","525,000.00","525,000.00",0.00,"1,132,812.50"


In [43]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   54,760.42               54,760.42   
     2                   49,000.00               49,000.00   
     3                   54,250.00               54,250.00   
     4                   52,500.00               52,500.00   
     5                   54,250.00               54,250.00   
...                            ...                     ...   
2012 9                   52,500.00               52,500.00   
     10                  54,250.00               54,250.00   
     11                  52,500.00               52,500.00   
     12                  54,250.00               54,250.00   
2013 1                        0.00                    0.00   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   54,760.42                  0.00               0.00  
     2                   49,000.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   52,500.00                  0.00               0.00  
     5                   54,250.00             75,180.41               0.00  
...                            ...                   ...                ...  
2012 9                   52,500.00                  0.00               0.00  
     10                  54,250.00                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  54,250.00                  0.00       3,950,177.08  
2013 1                        0.00                  0.00         150,000.00  

[121 rows x 5 columns]

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [44]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.89,0.96


In [45]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.85,0.93
2004,1.00,1.00,1.00,0.87,0.97
2005,1.00,1.00,1.00,0.83,0.91
2006,1.00,1.00,1.00,0.83,0.97
2007,1.00,1.00,1.00,0.95,0.98
2008,1.00,1.00,1.00,0.00,0.91
2009,1.00,1.00,1.00,0.93,0.97
2010,1.00,1.00,1.00,0.00,0.95
2011,1.00,1.00,1.00,0.85,0.97


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [46]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"121,407.22"


In [47]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Heavy Lift Vessel,Field Support Vessel
0,2003,"11,218.25","2,159.00","2,223.75","2,141.75","3,589.75","1,104.00"
1,2004,"16,369.19","3,748.00","3,690.50","3,840.00","3,955.69","1,135.00"
2,2005,"12,581.25","3,209.50","3,243.25","3,213.50","1,811.00","1,104.00"
3,2006,"14,353.71","2,724.50","2,645.75","2,733.00","5,708.96",541.50
4,2007,"11,612.49","2,889.25","2,863.75","2,910.75","2,235.24",713.50
5,2008,"9,484.29","2,441.00","2,529.75","2,811.25","1,702.29",0.00
6,2009,"11,896.16","2,265.50","2,472.25","2,557.75","3,452.66","1,148.00"
7,2010,"9,560.38","2,561.00","2,545.00","2,515.00","1,939.38",0.00
8,2011,"12,274.24","2,540.00","2,564.00","2,513.50","3,552.74","1,104.00"
9,2012,"12,057.27","2,659.50","2,422.25","2,495.25","3,871.27",609.00


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [48]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [49]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [50]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [51]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [52]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [53]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
...                      ...
2012 9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00
2013 1                  0.00

[121 rows x 1 columns]

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [54]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,145,372.40"
Repair,"23,794,806.42"
Crew Transfer,"1,197,765.63"
Site Travel,0.00
Mobilization,"8,500,000.00"
Weather Delay,"34,584,818.58"
No Requests,"19,047,042.00"
Not in Shift,"21,465,343.76"


In [55]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,145,372.40","2,145,372.40"
Repair,0.00,0.00,0.00,"23,794,806.42","23,794,806.42"
Crew Transfer,0.00,0.00,0.00,"1,197,765.63","1,197,765.63"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"8,500,000.00","8,500,000.00"
Weather Delay,0.00,0.00,0.00,"34,584,818.58","34,584,818.58"
No Requests,0.00,0.00,0.00,"19,047,042.00","19,047,042.00"
Not in Shift,0.00,0.00,0.00,"21,465,343.76","21,465,343.76"


In [56]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False)

total_cost
year reason                    
2003 Maintenance      91,747.40
     Repair        2,309,925.35
     Crew Transfer   110,703.13
     Site Travel           0.00
     Mobilization  1,000,000.00
...                         ...
2013 Site Travel           0.00
     Mobilization          0.00
     Weather Delay   150,000.00
     No Requests           0.00
     Not in Shift          0.00

[88 rows x 1 columns]

In [57]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False)

total_cost
month reason                    
1     Maintenance     151,466.15
      Repair        3,801,934.90
      Crew Transfer   167,591.15
      Site Travel           0.00
      Mobilization  1,500,000.00
...                          ...
12    Site Travel           0.00
      Mobilization  1,500,000.00
      Weather Delay 5,212,104.17
      No Requests     930,955.73
      Not in Shift  1,520,349.84

[96 rows x 1 columns]

In [58]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          22,731.77
           Crew Transfer    2,570.31
           Site Travel          0.00
           Mobilization         0.00
...                              ...
2013 12    Site Travel          0.00
           Mobilization         0.00
           Weather Delay        0.00
           No Requests          0.00
           Not in Shift         0.00

[1056 rows x 1 columns]

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [59]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
turbine,"122,300,801.83"


In [60]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
turbine   delay                  0.00              0.00   55,914,857.38   
          maintenance            0.00              0.00    2,145,372.40   
          repair                 0.00              0.00   23,794,806.42   

                         total_cost  
component action                     
turbine   delay       55,914,857.38  
          maintenance  2,145,372.40  
          repair      23,794,806.42

In [61]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
year,component,,,,
2003,turbine,"3,376,000.00",0.00,"7,421,474.83","10,797,474.83"
2004,turbine,"5,524,000.00",0.00,"12,649,081.99","18,173,081.99"
2005,turbine,"3,907,000.00",0.00,"5,044,039.93","8,951,039.93"
2006,turbine,"5,342,500.00",0.00,"13,265,091.66","18,607,591.66"
2007,turbine,"4,203,000.00",0.00,"9,241,009.05","13,444,009.05"
2008,turbine,"2,804,000.00",0.00,"4,440,907.12","7,244,907.12"
2009,turbine,"3,683,500.00",0.00,"6,334,532.12","10,018,032.12"
2010,turbine,"3,046,500.00",0.00,"6,019,978.30","9,066,478.30"
2011,turbine,"3,448,000.00",0.00,"9,247,994.13","12,695,994.13"


In [62]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

,,materials_cost,total_labor_cost,equipment_cost,total_cost
month,component,,,,
1,turbine,"4,955,500.00",0.00,"15,810,463.54","20,765,963.54"
2,turbine,"2,994,000.00",0.00,"4,219,935.97","7,213,935.97"
3,turbine,"1,281,500.00",0.00,"1,675,501.74","2,957,001.74"
4,turbine,"3,925,000.00",0.00,"9,534,105.04","13,459,105.04"
5,turbine,"7,449,500.00",0.00,"13,528,997.79","20,978,497.79"
6,turbine,"3,455,500.00",0.00,"5,482,774.31","8,938,274.31"
7,turbine,"3,297,500.00",0.00,"4,317,434.90","7,614,934.90"
8,turbine,"3,057,000.00",0.00,"5,369,696.18","8,426,696.18"
9,turbine,"1,661,000.00",0.00,"2,058,835.94","3,719,835.94"


In [63]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1     turbine         55,000.00              0.00       49,401.04   
     2     turbine         58,000.00              0.00       52,117.19   
     3     turbine         57,000.00              0.00       54,395.83   
     4     turbine        108,500.00              0.00       41,580.73   
     5     turbine        320,500.00              0.00      117,953.13   
     6     turbine         78,500.00              0.00      848,494.79   
     7     turbine      1,712,000.00              0.00    3,410,963.54   
     8     turbine        101,000.00              0.00       77,018.23   
     9     turbine         76,500.00              0.00       50,731.77   
     10    turbine        178,500.00              0.00      149,351.56   
     11    turbine        168,000.00              0.00       97,671.88   
     12    turbine        462,500.00              0.00    2,471,795.14   
2004 1     turbine        890,500.00              0.00    3,203,387.15   
     2     turbine        141,500.00              0.00      148,750.00   
     3     turbine        210,000.00              0.00      161,000.00   
     4     turbine        272,000.00              0.00      157,500.00   
     5     turbine      1,388,500.00              0.00    2,398,160.98   
     6     turbine        255,500.00              0.00      115,575.52   
     7     turbine        130,000.00              0.00       67,940.10   
     8     turbine        162,000.00              0.00      104,052.08   
     9     turbine        184,500.00              0.00      134,294.27   
     10    turbine        571,500.00              0.00    1,890,802.08   
     11    turbine      1,152,000.00              0.00    4,104,869.79   
     12    turbine        166,000.00              0.00      162,750.00   
2005 1     turbine        185,500.00              0.00      161,000.00   
     2     turbine        134,000.00              0.00      144,375.00   
     3     turbine        147,500.00              0.00      105,473.96   
     4     turbine         70,500.00              0.00       50,859.38   

                       total_cost  
year month component               
2003 1     turbine     104,401.04  
     2     turbine     110,117.19  
     3     turbine     111,395.83  
     4     turbine     150,080.73  
     5     turbine     438,453.13  
     6     turbine     926,994.79  
     7     turbine   5,122,963.54  
     8     turbine     178,018.23  
     9     turbine     127,231.77  
     10    turbine     327,851.56  
     11    turbine     265,671.88  
     12    turbine   2,934,295.14  
2004 1     turbine   4,093,887.15  
     2     turbine     290,250.00  
     3     turbine     371,000.00  
     4     turbine     429,500.00  
     5     turbine   3,786,660.98  
     6     turbine     371,075.52  
     7     turbine     197,940.10  
     8     turbine     266,052.08  
     9     turbine     318,794.27  
     10    turbine   2,462,302.08  
     11    turbine   5,256,869.79  
     12    turbine     328,750.00  
2005 1     turbine     346,500.00  
     2     turbine     278,375.00  
     3     turbine     252,973.96  
     4     turbine     121,359.38

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [64]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [65]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [66]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [67]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,...,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,"16,013,230.75"


In [68]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [69]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [75]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low")

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
...                ...
2012 8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09

[120 rows x 1 columns]

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [76]:
metrics.opex("project")

,OpEx
0,"165,996,379.54"


In [77]:
metrics.opex("annual")

,OpEx
year,
2003,"14,894,186.67"
2004,"23,475,960.55"
2005,"12,519,767.95"
2006,"22,568,964.25"
2007,"16,665,089.96"
2008,"11,442,645.06"
2009,"17,429,969.83"
2010,"12,144,790.99"
2011,"17,782,624.52"


In [78]:
metrics.opex("monthly")

,OpEx
month,
1,"24,460,650.49"
2,"11,218,125.53"
3,"5,465,667.85"
4,"20,138,604.61"
5,"24,623,583.89"
6,"15,081,295.89"
7,"11,102,910.90"
8,"10,993,591.46"
9,"5,683,887.57"


In [79]:
metrics.opex("month-year")

OpEx
year month           
2003 1     355,172.34
     2     327,740.34
     3     355,641.09
     4     397,507.51
     5     694,321.50
...               ...
2013 8           0.00
     9           0.00
     10          0.00
     11          0.00
     12          0.00

[132 rows x 1 columns]

In [80]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
...               ...
2012 9              0
     10             0
     11             0
     12             0
2013 1              0

[121 rows x 1 columns]

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [81]:
# Project totals at the project level
total = metrics.process_times()
total

/Users/rhammond/Documents/GitHub/WOMBAT/wombat/core/post_processor.py:1754: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  process_single = partial(_process_single, events)


,time_to_completion,process_time,downtime,N
category,,,,
annual service,"179,110.80","69,178.25","62,541.25",492.00
major repair,"81,892.73","1,846.10","1,389.81",31.00
major replacement,"171,090.85","11,458.39","171,090.85",68.00
manual reset,"171,891.68","36,574.50","14,804.00","5,692.00"
medium repair,"12,456.97","10,110.00","8,090.25",214.00
minor repair,"60,980.74","30,977.75","15,830.00","2,236.00"


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [82]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

/Users/rhammond/Documents/GitHub/WOMBAT/wombat/core/post_processor.py:1812: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,windfarm
Project Energy Production (kWh),"9,639,053,164.50"


In [83]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

/Users/rhammond/Documents/GitHub/WOMBAT/wombat/core/post_processor.py:1812: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


,windfarm,S00T1,S00T2,S00T3,S00T4,...,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,639,053,164.50","110,293,643.50","118,918,659.50","122,710,249.00","121,660,839.00",...,"116,916,521.50","122,653,962.50","122,088,967.50","122,834,446.00","106,751,513.50"


In [84]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"847,619,445.50"
2004,"934,215,733.00"
2005,"986,381,500.00"
2006,"947,356,160.00"
2007,"1,034,009,887.50"
2008,"1,056,948,280.00"
2009,"962,611,969.00"
2010,"857,735,629.00"
2011,"1,021,160,049.00"


In [85]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,074,470,433.50"
2,"823,411,948.00"
3,"898,918,028.50"
4,"705,201,314.50"
5,"691,411,930.50"
6,"561,022,725.00"
7,"592,332,889.50"
8,"644,255,742.00"
9,"794,370,125.00"


In [87]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False)

windfarm
year month               
2003 1     111,754,613.00
     2      58,290,600.50
     3      65,858,903.00
     4      90,362,816.50
     5      59,848,567.00
...                   ...
2012 8      54,977,094.50
     9      94,027,326.00
     10     91,765,963.00
     11     89,758,331.00
     12    106,554,560.00

[120 rows x 1 columns]

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [88]:
metrics.npv("project")

,NPV
0,"187,532,698.93"


In [89]:
metrics.opex("annual")

,OpEx
year,
2003,"14,894,186.67"
2004,"23,475,960.55"
2005,"12,519,767.95"
2006,"22,568,964.25"
2007,"16,665,089.96"
2008,"11,442,645.06"
2009,"17,429,969.83"
2010,"12,144,790.99"
2011,"17,782,624.52"


In [90]:
metrics.opex("monthly")

,OpEx
month,
1,"24,460,650.49"
2,"11,218,125.53"
3,"5,465,667.85"
4,"20,138,604.61"
5,"24,623,583.89"
6,"15,081,295.89"
7,"11,102,910.90"
8,"10,993,591.46"
9,"5,683,887.57"


In [97]:
metrics.opex("month-year")

OpEx
year month           
2003 1     355,172.34
     2     327,740.34
     3     355,641.09
     4     397,507.51
     5     694,321.50
...               ...
2013 8           0.00
     9           0.00
     10          0.00
     11          0.00
     12          0.00

[132 rows x 1 columns]

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [92]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [93]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [94]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [95]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [96]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
